# Big Shadow State Tomography

In [1]:
import numpy as np

from time import time
from tqdm import tqdm

import qiskit

from qiskit_aer import AerSimulator
from qiskit.visualization import plot_gate_map 

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_ibm_runtime.fake_provider import FakeMontrealV2
from qiskit_ibm_runtime.fake_provider import FakeKyiv

from qiskit.circuit.random import random_circuit

import matplotlib.pyplot as plt

plt.style.use("dark_background")

%cd -q ../..

from rivet_transpiler import transpile_right

In [2]:
FAKE_BACKEND = FakeKyiv()

backend = AerSimulator.from_backend(FAKE_BACKEND)

backend.options.noise_model = None

# qiskit.visualization.plot_gate_map(FAKE_BACKEND)

#### 3.3. Functions

In [4]:
def rotate_qubit_to_basis(circuit, qubit, basis):

    if basis == "X":
        circuit.h(qubit)

    elif basis == "Y":
        circuit.sdg(qubit)
        circuit.h(qubit)

    elif basis == "Z":
        pass

In [5]:
def build_bases_rotation_circuit(bases):

    qubits_count = len(bases)

    circuit = qiskit.QuantumCircuit(qubits_count)

    for qubit, basis in enumerate(bases):

        if basis == "X":
            circuit.h(qubit)

        elif basis == "Y":
            circuit.sdg(qubit)
            circuit.h(qubit)

        elif basis == "Z":
            pass

    return circuit

In [6]:
def get_inverted_channel(channel, qubits_count):

    identity = np.eye(2 ** qubits_count)

    inverted_channel = (2 ** qubits_count + 1) * channel - identity

    return inverted_channel

In [7]:
def get_basis_unitary(basis):

    if basis == "X":
        unitary = 1/np.sqrt(2) * np.array([[1.,1.],
                                           [1.,-1.]])
    elif basis == "Y":
        unitary = 1/np.sqrt(2) * np.array([[1.,-1.0j],
                                           [1.,1.j]])
    elif basis == "Z":
        unitary = np.eye(2)

    return unitary

### Circuit

In [33]:
# Parameters

QUBITS_COUNT = 4
CIRCUIT_DEPTH = 100

OPTIMIZATION_LEVEL = 3

SEED = 1234

rng = np.random.default_rng(SEED) 

qubits = list(range(QUBITS_COUNT))

## Random Circuit

circuit = random_circuit(
    QUBITS_COUNT, 
    depth=CIRCUIT_DEPTH,
    seed=SEED,
    max_operands=1, 
    measure=False)

circuit.draw(fold=-1)

┌───┐     ┌───────────┐ ┌──────────────────────────┐       ┌─────────────┐                  ┌─────┐                      ┌───┐                 ┌──────────┐        ┌──────┐              ┌─────┐            ┌───┐    ┌────────────┐   ┌─────────────────┐┌───────────────────┐┌────────────┐     ┌──────────────────┐           ┌───┐              ┌──────┐                 ┌───┐           ┌────────────┐┌────────────────────────────┐ ┌───┐  ┌────────────┐┌─────┐  ┌──────────────────────────┐             ┌───┐                        ┌───┐                       ┌────┐               ┌───┐     ┌───────────────────┐ ┌──────┐          ┌─────┐                    ┌──────┐              ┌───┐             ┌───┐                   ┌───┐            ┌────────────┐┌─────────────────────────┐┌─────────────┐   ┌────────────┐              ┌────┐              ┌───────────────────┐    ┌────┐     ┌───┐       ┌───┐             ┌───┐                   ┌───┐            ┌───┐     ┌───────────────────┐      ┌─────┐          ┌────────────────────┐       ┌───┐     ┌────────────────────────────┐┌─────────────────┐           ┌───┐               ┌──────┐   ┌───────────────────────────┐   ┌─────┐         ┌─────┐      ┌───────────────────────────┐ ┌───┐ ┌───────────────────┐    ┌───┐     ┌─────────────┐           ┌───┐            ┌───────────────────────────┐┌──────────────────┐┌─────────────┐    ┌───┐              ┌───┐                 ┌───┐        ┌────────────┐┌───┐   ┌─────┐               ┌───┐              ┌───────────┐       ┌───┐               ┌─────┐           ┌────────────────────┐       ┌────────────┐                  ┌───┐                ┌────┐            ┌───┐        ┌──────────────────────────┐┌──────────────────┐           ┌───┐                  ┌────────────┐              ┌───┐        ┌───────────────────────────┐   ┌─────┐    ┌─────────────────┐     ┌───┐                ┌────┐                      ┌───┐                ┌───┐     ┌───────────────────┐        ┌───┐          ┌────────────┐           ┌───┐           ┌──────────────────────────┐ ┌───┐ 
q_0: ────┤ Z ├─────┤ P(2.0014) ├─┤ U3(4.2404,4.1461,4.6229) ├───────┤ U1(0.37785) ├──────────────────┤ Tdg ├──────────────────────┤ Z ├─────────────────┤ P(2.328) ├────────┤ √Xdg ├──────────────┤ Tdg ├────────────┤ S ├────┤ Ry(2.0514) ├───┤ U2(1.3833,4.28) ├┤ U2(3.1309,1.9145) ├┤ Rz(5.4904) ├─────┤ U2(1.896,1.0986) ├───────────┤ S ├──────────────┤ √Xdg ├─────────────────┤ Y ├───────────┤ Rx(4.5677) ├┤ U3(0.61464,0.88779,6.0462) ├─┤ X ├──┤ Rx(4.1755) ├┤ Tdg ├──┤ U(0.76705,3.3373,5.7374) ├─────────────┤ Z ├────────────────────────┤ Y ├───────────────────────┤ √X ├───────────────┤ T ├─────┤ U2(5.0853,5.0693) ├─┤ √Xdg ├──────────┤ Tdg ├────────────────────┤ √Xdg ├──────────────┤ T ├─────────────┤ I ├───────────────────┤ S ├────────────┤ Rz(4.3659) ├┤ U(3.1414,3.3284,6.1928) ├┤ Rz(0.02456) ├───┤ Rx(4.0116) ├──────────────┤ √X ├──────────────┤ U2(2.9062,2.7859) ├────┤ √X ├─────┤ I ├───────┤ X ├─────────────┤ S ├───────────────────┤ Y ├────────────┤ I ├─────┤ U2(1.1211,5.7736) ├──────┤ Tdg ├──────────┤ U2(2.9712,0.65438) ├───────┤ X ├─────┤ U3(2.2801,0.085473,1.4678) ├┤ R(2.2482,2.969) ├───────────┤ Z ├───────────────┤ √Xdg ├───┤ U3(2.3508,4.6039,0.58421) ├───┤ Sdg ├─────────┤ Tdg ├──────┤ U3(4.7452,0.39556,4.6099) ├─┤ S ├─┤ U2(5.1244,1.6147) ├────┤ T ├─────┤ Ry(0.92448) ├───────────┤ X ├────────────┤ U3(5.1578,0.81909,4.9134) ├┤ R(5.5004,4.5534) ├┤ Rx(0.36878) ├────┤ Z ├──────────────┤ X ├─────────────────┤ I ├────────┤ Ry(5.2862) ├┤ I ├───┤ Sdg ├───────────────┤ T ├──────────────┤ P(3.6441) ├───────┤ S ├───────────────┤ Tdg ├───────────┤ U2(0.36025,6.1273) ├───────┤ Rz(4.1198) ├──────────────────┤ T ├────────────────┤ √X ├────────────┤ S ├────────┤ U3(3.4607,1.3813,3.4414) ├┤ R(5.7271,5.5177) ├───────────┤ X ├──────────────────┤ Rx(2.9678) ├──────────────┤ H ├────────┤ U3(0.73263,2.0271,3.5123) ├───┤ Sdg ├────┤ R(5.4782,3.524) ├─────┤ I ├────────────────┤ √X ├──────────────────────┤ Z ├────────────────┤ Z ├─────┤ U2(2.0

### 4.2.  Run Paulis Experiment

In [34]:
TOTAL_SHOTS_COUNT = 10000

In [35]:
random_bases = rng.choice(['X','Y','Z'], size=(TOTAL_SHOTS_COUNT, QUBITS_COUNT))

bases_list, shots_counts = np.unique(random_bases, axis=0, return_counts=True)

bases_list.shape

(81, 4)

In [36]:
rotation_circuits = []

for bases in bases_list:

    rotation_circuit = build_bases_rotation_circuit(bases)

    rotation_circuits.append(rotation_circuit)

len(rotation_circuits)

81

#### Basic Transpilation - Slower Version!


In [37]:
start_time = time()

# Compose and Transpile all Rotation Circuits

shadow_circuits = []

rotation_tracker = tqdm(rotation_circuits, unit=' basis', ncols=110)

for rotation_circuit in rotation_tracker:

    measurement_circuit = rotation_circuit.measure_all(inplace=False)

    measured_circuit = circuit.compose(measurement_circuit)

    # display(measured_circuit.draw(fold=-1))

    shadow_circuit = qiskit.transpile(
        measured_circuit,
        backend,
        seed_transpiler=SEED,
        optimization_level=OPTIMIZATION_LEVEL)

    shadow_circuits.append(shadow_circuit)

estimated_time = time() - start_time

print(f"estimated_time: {estimated_time:.02f}")

100%|█████████████████████████████████████████████████████████████████████| 81/81 [00:07<00:00, 10.59 basis/s]

estimated_time: 7.65


#### Transpile Right - Much Faster!

In [38]:
start_time = time()

# Transpile Central Part

transpiled_central_circuit = qiskit.transpile(
    circuit,
    backend,
    seed_transpiler=SEED,
    optimization_level=OPTIMIZATION_LEVEL)


# Transpile Right all Rotation Circuits

shadow_circuits = []

rotation_tracker = tqdm(rotation_circuits, unit=' basis', ncols=110)

for rotation_circuit in rotation_tracker:

    measurement_circuit = rotation_circuit.measure_all(inplace=False)

    shadow_circuit = transpile_right(
        transpiled_central_circuit,
        measurement_circuit,
        backend=backend,
        seed_transpiler=SEED,
        optimization_level=OPTIMIZATION_LEVEL)

    shadow_circuits.append(shadow_circuit)


estimated_time = time() - start_time

print(f"estimated_time: {estimated_time:.02f}")

100%|█████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 38.97 basis/s]

estimated_time: 2.18
